In [8]:
pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install python-dotenv mysql-connector-python

Note: you may need to restart the kernel to use updated packages.


In [1]:
import mysql.connector
from mysql.connector import errorcode

import os
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
# Crea la base de datos 'HR'
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',     
)
cur = cnx.cursor()

try:
    cur.execute("CREATE DATABASE HR")
    print("Base de datos 'HR' creada exitosamente.")
except mysql.connector.Error as err:
    print(err)
    print("Código de Error:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Mensaje", err.msg)

cnx.close()
# Si ejecutas esta celda dos veces, obtendrás un aviso de que la base de datos ya existe.


1007 (HY000): Can't create database 'HR'; database exists
Código de Error: 1007
SQLSTATE HY000
Mensaje Can't create database 'HR'; database exists


In [4]:
try:
    # Intenta hacer la conexión con la base de datos 'HR'
    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab',
        host='127.0.0.1',      # o la IP del servidor MySQL
        database='HR'
    )
    cur = cnx.cursor()
    
    cur.execute("""
    CREATE TABLE IF NOT EXISTS employees (
        employeenumber INTEGER PRIMARY KEY,
        gender VARCHAR(10),
        maritalstatus VARCHAR(20),
        attrition VARCHAR(10),
        datebirth INTEGER
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS satisfaction (
        employeenumber INTEGER,
        environmentsatisfaction INTEGER,
        jobsatisfaction INTEGER,
        relationshipsatisfaction INTEGER,
        jobinvolvement INTEGER,
        worklifebalance INTEGER,
        performancerating INTEGER,
        CONSTRAINT fk_satisfaction_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS work (
        employeenumber INTEGER,
        department VARCHAR(75),
        jobrole VARCHAR(75),
        joblevel INTEGER,
        remotework VARCHAR(10),
        standardhours VARCHAR(20),
        CONSTRAINT fk_work_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS history (
        employeenumber INTEGER,
        totalworkingyears FLOAT,
        yearsatcompany INTEGER,
        yearswithcurrmanager INTEGER,
        yearssincelastpromotion INTEGER,
        trainingtimeslastyear INTEGER,        
        CONSTRAINT fk_history_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS conditions (
        employeenumber INTEGER,
        monthlyincome FLOAT,
        monthlyrate FLOAT,
        dailyrate FLOAT,
        hourlyrate FLOAT,
        salary FLOAT,
        stockoptionlevel INTEGER,
        percentsalaryhike INTEGER,
        overtime VARCHAR(25),
        CONSTRAINT fk_conditions_emp FOREIGN KEY (employeenumber) REFERENCES employees(employeenumber)
    );
    """)

except mysql.connector.Error as err:
    # Si es un error de acceso denegado (ej. contraseña o usuario incorrecto)
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Algo está mal con tu nombre de usuario o contraseña.") #
    # Si la base de datos no existe
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("La base de datos no existe.") #
    # Para cualquier otro tipo de error
    else:
        print(err) #
        print("Código de Error:", err.errno) #
        print("SQLSTATE", err.sqlstate) #
        print("Mensaje", err.msg) #
else:
    # Si la conexión fue exitosa, cierra la conexión
    # cnx.close()
    print("Conexión cerrada.")

Conexión cerrada.


In [5]:
import pandas as pd
import mysql.connector

In [6]:
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database='HR'
)

mycursor = cnx.cursor() # Iniciamos el cursor

mycursor.execute("SHOW TABLES") # Ejecutamos la query

# El cursor se convierte en un iterable al que podemos acceder, por ejemplo, con un bucle for
print("Tablas en la base de datos 'HR':")
for x in mycursor:
    print(x) # Devuelve cada resultado como una tupla

cnx.close()

Tablas en la base de datos 'HR':
('conditions',)
('employees',)
('history',)
('satisfaction',)
('work',)


In [ ]:
# Cargar el CSV en un DataFrame
df = pd.read_csv('hr_clean_data_v6_BBDD.csv')

# Conexión a la base de datos MySQL
cnx = mysql.connector.connect(
    user='root',
    password='AlumnaAdalab',
    host='127.0.0.1',
    database='HR'
)
cur = cnx.cursor()

# Insertar los datos fila por fila en la tabla 'employees'
for index, row in df.iterrows():
    cur.execute("""
        INSERT INTO employees (
            employeenumber, gender, maritalstatus, attrition, datebirth
        ) VALUES (%s, %s, %s, %s, %s)
    """, (
        row['employeenumber'],
        row['gender'],
        row['maritalstatus'],
        row['attrition'],
        row['datebirth']
    ))
    
    cur.execute("""
        INSERT INTO satisfaction (
            employeenumber, environmentsatisfaction, jobsatisfaction,
            relationshipsatisfaction, jobinvolvement, worklifebalance,
            performancerating
        ) VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['employeenumber'],
        row['environmentsatisfaction'],
        row['jobsatisfaction'],
        row['relationshipsatisfaction'],
        row['jobinvolvement'],
        row['worklifebalance'],
        row['performancerating']
    ))

    cur.execute("""
        INSERT INTO work (
            employeenumber, department, jobrole, joblevel, remotework, standardhours
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['employeenumber'],
        row['department'],
        row['jobrole'],
        row['joblevel'],
        row['remotework'],
        row['standardhours']
    ))

    cur.execute("""
        INSERT INTO history (
            employeenumber, totalworkingyears, yearsatcompany,
            yearswithcurrmanager, yearssincelastpromotion,
            trainingtimeslastyear
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        row['employeenumber'],
        row['totalworkingyears'],
        row['yearsatcompany'],
        row['yearswithcurrmanager'],
        row['yearssincelastpromotion'],
        row['trainingtimeslastyear']
    ))

    cur.execute("""
        INSERT INTO conditions (
            employeenumber, monthlyincome, monthlyrate, dailyrate,
            hourlyrate, salary, stockoptionlevel, percentsalaryhike,
            overtime
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['employeenumber'],
        row['monthlyincome'],
        row['monthlyrate'],
        row['dailyrate'],
        row['hourlyrate'],
        row['salary'],
        row['stockoptionlevel'],
        row['percentsalaryhike'],
        row['overtime']
    ))

cnx.commit()
cur.close()
cnx.close()

In [12]:
try:
    df = pd.read_csv('hr_clean_data_v6_BBDD.csv')

    cnx = mysql.connector.connect(
        user='root',
        password='AlumnaAdalab',
        host='127.0.0.1',
        database='HR'
    )
    cur = cnx.cursor()

    for index, row in df.iterrows():
        cur.execute("""
            INSERT INTO employees (
                employeenumber, gender, maritalstatus, attrition, datebirth
            ) VALUES (%s, %s, %s, %s, %s)
        """, (
            row['employeenumber'],
            row['gender'],
            row['maritalstatus'],
            row['attrition'],
            row['datebirth']  # revisar formato aquí si da error
        ))

    cnx.commit()  # Confirmar cambios
    cur.close()
    cnx.close()
    print("Datos insertados correctamente")

except mysql.connector.Error as err:
    print(f"Error MySQL: {err}")
except Exception as e:
    print(f"Error general: {e}")

Error MySQL: 1205 (HY000): Lock wait timeout exceeded; try restarting transaction
